# Value Stock Recommender
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

In this project, I have built an investing strategy that selects the top 'x' stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 'x' stocks.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import yfinance as yf

# Reading Stock List

In [2]:
stocks = pd.read_csv('NIFTY_50_STOCKS.csv')

# Creating a Dictionary with stock information(stats)

For this Process, I have used yahoo finance Api to provide Financial Ratios.

In [3]:
company=yf.Ticker("TATAMOTORS.NS")
data=company.info

# Checking for Non-null Values

Some metrics on yahoo finance contain 'None' values.
These would obviously not be useful to us. 


The following code is to check what metrics are available to us.

In [4]:
for x in data:
    if(str(data[x])!='None'):
        print(x + ' : ' + str(data[x]))

zip : 400018
sector : Consumer Cyclical
fullTimeEmployees : 75278
longBusinessSummary : Tata Motors Limited designs, develops, manufactures, and sells a range of automotive vehicles. The company offers passenger cars; utility vehicles; intermediate and light commercial vehicles; small, medium, and heavy commercial vehicles; defense vehicles; pickups, wingers, buses, and trucks; and electric vehicles, as well as related spare parts and accessories. It also manufactures engines for industrial and marine applications; aggregates comprising axles and transmissions for commercial vehicles; and factory automation equipment, as well as provides information technology and vehicle financing services. The company offers its products under the Tata, Daewoo, Fiat, Jaguar, and Land Rover brands. It operates in India, China, the United States, the United Kingdom, rest of Europe, and internationally. The company was formerly known as Tata Engineering and Locomotive Company Limited and changed its nam

# Parsing required data

I have considered to following metrics to analyse the value of a company :
1. Price-to-Book Value
2. Price-to-Sales
3. EV/EBIDA
4. EV/Revenue
5. Earning Per Share
6. Percentage Held by Insiders

In [5]:

pb_ratio = data['priceToBook']

price_to_sales_ratio= data['priceToSalesTrailing12Months']

ev_to_ebitda = data['enterpriseToEbitda']

ev_to_revenue =data['enterpriseToRevenue']

eps =data['trailingEps']

held_percent_insiders=data['heldPercentInsiders']


 

# Creating a Dataframe



In [6]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/Revenue',
    'EV/Revenue Percentile',
    'Eps',
    'Eps Percentile',
    'HeldByInsiders(%)',
    'HeldByInsiders Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe



,Ticker,Price,Number of Shares to Buy,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,Eps,Eps Percentile,HeldByInsiders(%),HeldByInsiders Percentile,RV Score


# Entering values from the Dictionary

In [ ]:
for stock in stocks['Symbol']:
    company=yf.Ticker(stock + '.NS')
    
    rv_dataframe = rv_dataframe.append(
            pd.Series([
                stock,
                company.info['currentPrice'],
                'N/A',
                company.info['priceToBook'],
                'N/A',
                company.info['priceToSalesTrailing12Months'],
                'N/A',
                company.info['enterpriseToEbitda'],
                'N/A',
                company.info['enterpriseToRevenue'],
                'N/A',
                company.info['trailingEps'],
                'N/A',
                company.info['heldPercentInsiders'],
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
    
    
    
    

In [ ]:
rv_dataframe

# Checking for Null Values
There is missing data in our DataFrame.

The following steps written in code is to deal with the missing values and replace them with the mean of each column.  

In [11]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,Eps,Eps Percentile,HeldByInsiders(%),HeldByInsiders Percentile,RV Score
1,BAJFINANCE,6544.10,N/A,10.6633,N/A,22.4422,N/A,None,N/A,27.527,N/A,73.603,N/A,0.56690,N/A,N/A
2,M&M,791.35,N/A,None,N/A,1.30964,N/A,11.769,N/A,2.052,N/A,None,N/A,0.25603,N/A,N/A
9,HDFC,2710.75,N/A,3.12039,N/A,4.45812,N/A,None,N/A,6.192,N/A,111.356,N/A,0.01013,N/A,N/A
16,HDFCBANK,1513.00,N/A,3.80478,N/A,10.4003,N/A,None,N/A,10.986,N/A,59.298,N/A,0.21623,N/A,N/A
20,AXISBANK,754.40,N/A,2.18019,N/A,7.85527,N/A,None,N/A,11.040,N/A,27.772,N/A,0.02295,N/A,N/A
24,BAJFINANCE,6544.10,N/A,10.6633,N/A,22.4422,N/A,None,N/A,27.527,N/A,73.603,N/A,0.56690,N/A,N/A
25,M&M,791.35,N/A,None,N/A,1.30964,N/A,11.769,N/A,2.052,N/A,None,N/A,0.25603,N/A,N/A
32,HDFC,2457.25,N/A,2.82858,N/A,4.14848,N/A,None,N/A,5.904,N/A,104.7,N/A,0.01025,N/A,N/A
40,BAJFINANCE,6544.10,N/A,10.6633,N/A,22.4422,N/A,None,N/A,27.527,N/A,73.603,N/A,0.56690,N/A,N/A
41,M&M,791.35,N/A,None,N/A,1.30964,N/A,11.769,N/A,2.052,N/A,None,N/A,0.25603,N/A,N/A


In [12]:
for column in ['Price-to-Book Ratio', 'EV/EBITDA','EV/Revenue',  'Eps']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,Eps,Eps Percentile,HeldByInsiders(%),HeldByInsiders Percentile,RV Score
62,HINDALCO,426.95,N/A,1.427516,N/A,None,N/A,6.533,N/A,0.972,N/A,31.346,N/A,0.35559,N/A,N/A


# Calculating Percentile of each stock in each metric

In [14]:
#up_metrics is a list with metrics that are good indication for a company as they increase
up_metrics = {
            
            
            
            
            'Eps':'Eps Percentile',
            'HeldByInsiders(%)':'HeldByInsiders Percentile'
    
}

#down_metrics is a list with metrics that are good indication for a company as they decrease
down_metrics={
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/Revenue':'EV/Revenue Percentile'
}

for row in rv_dataframe.index:
    for metric in up_metrics.keys():
        rv_dataframe.loc[row, up_metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
        
    for metric in down_metrics.keys():
        rv_dataframe.loc[row, down_metrics[metric]] = (100-stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric]))/100

# Print each percentile score to make sure it was calculated properly
#for metric in metrics.values():
    #print(rv_dataframe[metric])

TypeError: '<' not supported between instances of 'NoneType' and 'float'

In [17]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,Eps,Eps Percentile,HeldByInsiders(%),HeldByInsiders Percentile,RV Score
0,TATASTEEL,1500.45,N/A,2.420096,0.72,0.979940,0.86,5.983000,0.92,1.397,0.84,177.356000,0.9,0.36043,0.32,N/A
1,BAJFINANCE,6544.10,N/A,10.663266,0.22,22.442215,0.04,96.727098,0.12,27.527,0.04,73.603000,0.68,0.56690,0.66,N/A
2,M&M,791.35,N/A,30.433027,0.06,1.309635,0.8,11.769000,0.7,2.052,0.78,79.023837,0.72,0.25603,0.16,N/A
3,BRITANNIA,3720.75,N/A,25.254189,0.08,6.831707,0.28,38.362000,0.36,6.872,0.28,70.886000,0.64,0.50547,0.46,N/A
4,IOC,106.20,N/A,0.871964,0.96,0.238657,0.98,4.110000,0.96,0.460,0.98,27.782000,0.32,0.73337,0.96,N/A
5,BPCL,463.50,N/A,1.728800,0.86,0.382140,0.92,6.447000,0.9,0.496,0.96,84.179000,0.74,0.53593,0.56,N/A
6,JSWSTEEL,737.45,N/A,3.793799,0.6,1.832601,0.7,7.947000,0.84,2.348,0.68,59.351000,0.6,0.65261,0.84,N/A
7,BAJAJFINSV,15006.35,N/A,6.665007,0.26,3.994201,0.48,14.118000,0.68,5.713,0.4,256.804000,0.98,0.68877,0.88,N/A
8,RELIANCE,2172.65,N/A,2.000180,0.8,2.832828,0.62,18.606000,0.54,3.036,0.62,73.252000,0.66,0.49596,0.42,N/A
9,HDFC,2710.75,N/A,3.120392,0.64,4.458124,0.46,96.727098,0.12,6.192,0.36,111.356000,0.84,0.01013,0.04,N/A


# Calculating the RV score
Rv score is the value score of the individual stock. The higher the better.

It is calculated by taking the mean of every Metric-Percentile for each stock. 

In [18]:
from statistics import mean

metrics={
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/Revenue':'EV/Revenue Percentile',
    'Eps':'Eps Percentile',
    'HeldByInsiders(%)':'HeldByInsiders Percentile'
    
}


for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,Eps,Eps Percentile,HeldByInsiders(%),HeldByInsiders Percentile,RV Score
0,TATASTEEL,1500.45,N/A,2.420096,0.72,0.979940,0.86,5.983000,0.92,1.397,0.84,177.356000,0.9,0.36043,0.32,0.76
1,BAJFINANCE,6544.10,N/A,10.663266,0.22,22.442215,0.04,96.727098,0.12,27.527,0.04,73.603000,0.68,0.56690,0.66,0.293333
2,M&M,791.35,N/A,30.433027,0.06,1.309635,0.8,11.769000,0.7,2.052,0.78,79.023837,0.72,0.25603,0.16,0.536667
3,BRITANNIA,3720.75,N/A,25.254189,0.08,6.831707,0.28,38.362000,0.36,6.872,0.28,70.886000,0.64,0.50547,0.46,0.35
4,IOC,106.20,N/A,0.871964,0.96,0.238657,0.98,4.110000,0.96,0.460,0.98,27.782000,0.32,0.73337,0.96,0.86
5,BPCL,463.50,N/A,1.728800,0.86,0.382140,0.92,6.447000,0.9,0.496,0.96,84.179000,0.74,0.53593,0.56,0.823333
6,JSWSTEEL,737.45,N/A,3.793799,0.6,1.832601,0.7,7.947000,0.84,2.348,0.68,59.351000,0.6,0.65261,0.84,0.71
7,BAJAJFINSV,15006.35,N/A,6.665007,0.26,3.994201,0.48,14.118000,0.68,5.713,0.4,256.804000,0.98,0.68877,0.88,0.613333
8,RELIANCE,2172.65,N/A,2.000180,0.8,2.832828,0.62,18.606000,0.54,3.036,0.62,73.252000,0.66,0.49596,0.42,0.61
9,HDFC,2710.75,N/A,3.120392,0.64,4.458124,0.46,96.727098,0.12,6.192,0.36,111.356000,0.84,0.01013,0.04,0.41


# Choosing top X stocks to invest in and creating a Portfolio

In this case the top 10 stocks have been chosen to create a Portfolio.

The Portfolio is an equal-weight Portfolio(every stock has the same weight).

In [19]:
rv_dataframe.sort_values(by = 'RV Score',ascending=False, inplace = True)
rv_dataframe = rv_dataframe[:10]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,Eps,Eps Percentile,HeldByInsiders(%),HeldByInsiders Percentile,RV Score
0,IOC,106.20,N/A,0.871964,0.96,0.238657,0.98,4.110,0.96,0.460,0.98,27.782,0.32,0.73337,0.96,0.86
1,BPCL,463.50,N/A,1.728800,0.86,0.382140,0.92,6.447,0.9,0.496,0.96,84.179,0.74,0.53593,0.56,0.823333
2,ONGC,113.20,N/A,0.644034,0.98,0.350578,0.96,4.323,0.94,0.662,0.94,17.693,0.18,0.70705,0.9,0.816667
3,COALINDIA,138.35,N/A,2.335691,0.74,0.958027,0.88,3.415,0.98,0.774,0.9,22.383,0.22,0.66134,0.86,0.763333
4,TATASTEEL,1500.45,N/A,2.420096,0.72,0.979940,0.86,5.983,0.92,1.397,0.84,177.356,0.9,0.36043,0.32,0.76
5,GRASIM,1501.20,N/A,1.505293,0.88,1.186386,0.82,9.409,0.78,2.315,0.72,87.248,0.76,0.41117,0.4,0.726667
6,JSWSTEEL,737.45,N/A,3.793799,0.6,1.832601,0.7,7.947,0.84,2.348,0.68,59.351,0.6,0.65261,0.84,0.71
7,HINDALCO,426.95,N/A,1.427516,0.9,0.643219,0.9,6.533,0.88,0.972,0.88,31.346,0.38,0.35559,0.3,0.706667
8,HEROMOTOCO,2763.60,N/A,3.581328,0.62,1.648560,0.72,11.210,0.72,1.476,0.82,155.907,0.86,0.36523,0.34,0.68
9,BAJAJ-AUTO,3833.05,N/A,4.067522,0.52,3.335588,0.56,14.730,0.66,3.073,0.6,194.663,0.92,0.58424,0.72,0.663333


In [20]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [21]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\nilay\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,Eps,Eps Percentile,HeldByInsiders(%),HeldByInsiders Percentile,RV Score
0,IOC,106.20,941,0.871964,0.96,0.238657,0.98,4.110,0.96,0.460,0.98,27.782,0.32,0.73337,0.96,0.86
1,BPCL,463.50,215,1.728800,0.86,0.382140,0.92,6.447,0.9,0.496,0.96,84.179,0.74,0.53593,0.56,0.823333
2,ONGC,113.20,883,0.644034,0.98,0.350578,0.96,4.323,0.94,0.662,0.94,17.693,0.18,0.70705,0.9,0.816667
3,COALINDIA,138.35,722,2.335691,0.74,0.958027,0.88,3.415,0.98,0.774,0.9,22.383,0.22,0.66134,0.86,0.763333
4,TATASTEEL,1500.45,66,2.420096,0.72,0.979940,0.86,5.983,0.92,1.397,0.84,177.356,0.9,0.36043,0.32,0.76
5,GRASIM,1501.20,66,1.505293,0.88,1.186386,0.82,9.409,0.78,2.315,0.72,87.248,0.76,0.41117,0.4,0.726667
6,JSWSTEEL,737.45,135,3.793799,0.6,1.832601,0.7,7.947,0.84,2.348,0.68,59.351,0.6,0.65261,0.84,0.71
7,HINDALCO,426.95,234,1.427516,0.9,0.643219,0.9,6.533,0.88,0.972,0.88,31.346,0.38,0.35559,0.3,0.706667
8,HEROMOTOCO,2763.60,36,3.581328,0.62,1.648560,0.72,11.210,0.72,1.476,0.82,155.907,0.86,0.36523,0.34,0.68
9,BAJAJ-AUTO,3833.05,26,4.067522,0.52,3.335588,0.56,14.730,0.66,3.073,0.6,194.663,0.92,0.58424,0.72,0.663333


# Exporting Portfolio to Excel
This is done using XLSxWriter

In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'₹0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [24]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Book Ratio',
       'PB Percentile', 'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/Revenue', 'EV/Revenue Percentile', 'Eps',
       'Eps Percentile', 'HeldByInsiders(%)', 'HeldByInsiders Percentile',
       'RV Score'],
      dtype='object')

In [25]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Book Ratio', float_template],
                    'E': ['PB Percentile', percent_template],
                    'F': ['Price-to-Sales Ratio', float_template],
                    'G': ['PS Percentile',percent_template],
                    'H': ['EV/EBITDA', float_template],
                    'I': ['EV/EBITDA Percentile', percent_template],
                    'J': ['EV/Revenue', float_template],
                    'K': ['EV/Revenue Percentile', percent_template],
                    'L': ['Eps', float_template],
                    'M': ['Eps Percentile', percent_template],
                    'N': ['HeldByInsiders(%)', float_template],
                    'O': ['HeldByInsiders(%) Percentile', percent_template],
                    'P': ['RV Score', float_template],
                    
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [26]:
writer.save()

The Portfolio will be saved in the same file as this notebook.